### Required Libraries

### Load Dataset

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
model_data.shape

(47632, 2832)

In [5]:
def map_weight(value):
    if (value >= '2022-01') and (value <= '2022-06'):
        return 2
    elif (value >= '2021-07') and (value <= '2021-12'):
        return 1
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


### Drop Null target records

In [6]:
model_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [7]:
df = model_data.dropna(subset=['G15_B24'])

In [8]:
df['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [9]:
df['user_segment'].value_counts()

user_segment
OFFUS    33813
ONUS     13819
Name: count, dtype: int64

In [10]:
# take one copy for model development data
data = df.copy()
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
data.shape

(47632, 2833)

### Split the Time interval

In [12]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [13]:
model_df1.shape, oot_df1.shape

((34344, 2833), (10802, 2833))

In [14]:
data['num_CONSUMER LOAN_enq_last_6M_12M'] = (data['num_CONSUMER LOAN_enq_last_6M']+data['num_CONSUMER LOAN_enq_last_36M'])/2

In [15]:
features_list = ['Num_Enq_all_loans_last_9M',
'Num_of_PERSONAL LOAN_opened_last9M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_36M',
'num_CREDIT CARD_enq_last_18M',
'Total_outstanding_Balance_CREDIT CARD',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'num_PL_LE_1_Lac_opened_in_last_12M',
'num_PL_LE_1_Lac_with_no_delinquency',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'Utilization_all',
'num_of_loans_le_1lac_non_credit',
                 
# Reconcilated Features
'percentage_ever_delinquent',
'months_since_most_recent_delinquency_installment',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'num_debits_gt_10000_M3',
'num_credit_gt_salary',
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',
'cash_withdrawal_sum',
'num_cash_withdrawal_last_1_months',
'bounced_transactions_count',
'min_balance_last_3_months',
'days_balance_lt_5000_M2',
'days_balance_lt_10000_M3',
'avg_num_credits_last_2_months',
'Max_Credit_Avg_M1_M2_M3',
'ratio_3th_day_balance_to_max_credit_M2',
'ratio_active_loan_vs_salary',
'num_self_trans_last_3_months',


'deviation_from_mean_income#last_30_Days',
'age',

'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 

'dm206s',
'g406s',
'balmag01',
'aggs907',

'Weight_vintage',          
'month_year',
'G15_B24']

In [16]:
len(features_list)

49

In [17]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [18]:
oot_df = oot_df1[features_list]

In [19]:
model_df.shape, oot_df.shape

((34344, 49), (10802, 49))

In [20]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

In [21]:
X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

In [22]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3040923/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [23]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3040923/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [24]:
X_oot.shape,y_oot.shape

((10802, 47), (10802,))

### Split Model dataset into Train and Test

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [26]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 47) (4122, 47) (13738, 47)


In [27]:
X_train.shape, X_test.shape

((20606, 47), (13738, 47))

In [28]:
#Get vinatge weight values 
train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values
oot_weights = X_oot['Weight_vintage'].values

In [29]:
#remove weight vinate from features 

In [30]:
X_train_new = X_train_new.drop(columns=['Weight_vintage'])

In [31]:
X_eval = X_eval.drop(columns=['Weight_vintage'])

In [32]:
X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])

In [33]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [34]:

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 5,
    'alpha': 4.5,
    'gamma': 4.8,
    'random_state': 42
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 300,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.58438	eval-auc:0.54811
[10]	train-auc:0.64663	eval-auc:0.63181
[20]	train-auc:0.65132	eval-auc:0.63301
[30]	train-auc:0.65164	eval-auc:0.63353
[40]	train-auc:0.65338	eval-auc:0.63273
[50]	train-auc:0.65409	eval-auc:0.63714
[60]	train-auc:0.65445	eval-auc:0.63873
[70]	train-auc:0.65889	eval-auc:0.64556
[80]	train-auc:0.66202	eval-auc:0.64450
[90]	train-auc:0.66302	eval-auc:0.64771
[100]	train-auc:0.66486	eval-auc:0.64847
[110]	train-auc:0.66630	eval-auc:0.64967
[120]	train-auc:0.66793	eval-auc:0.64975
[130]	train-auc:0.66892	eval-auc:0.65159
[140]	train-auc:0.67060	eval-auc:0.65246
[150]	train-auc:0.67136	eval-auc:0.65503
[160]	train-auc:0.67207	eval-auc:0.65513
[170]	train-auc:0.67241	eval-auc:0.65627
[180]	train-auc:0.67344	eval-auc:0.65812
[190]	train-auc:0.67399	eval-auc:0.65905
[200]	train-auc:0.67476	eval-auc:0.66098
[210]	train-auc:0.67580	eval-auc:0.66221
[220]	train-auc:0.67639	eval-auc:0.66312
[230]	train-auc:0.67711	eval-auc:0.66475
[240]	train-auc:0.67801	eva

In [35]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.68, Gini: 0.35, KS: 0.25
Eval AUC: 0.66, Gini: 0.33, KS: 0.26
Test AUC: 0.64, Gini: 0.29, KS: 0.22
OOT AUC: 0.65, Gini: 0.29, KS: 0.21


In [36]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.161887         0.135808        0.251853        0.112779   1081   
9          0.103704         0.101228        0.112720        0.090816   1080   
8          0.097222         0.083453        0.090808        0.076940   1080   
7          0.082407         0.072126        0.076919        0.068316   1080   
6          0.082407         0.066070        0.068308        0.064197   1080   
5          0.056481         0.062404        0.064196        0.060599   1080   
4          0.064815         0.057967        0.060597        0.055390   1080   
3          0.055556         0.053421        0.055381        0.051775   1080   
2          0.031481         0.050170        0.051771        0.048393   1080   
1          0.024977         0.045860        0.048387        0.041301   1081   

        label_good  label_bad  
Decile                         
10             906        175  
9              968        112  
8              975        105  
7              991         89  
6              991         89  
5             1019         61  
4             1010         70  
3             1020         60  
2             1046         34  
1             1054         27

In [37]:
xgb_model.save_model('LeoPayu_BBILAP_weightVintage_15_in_24mob.json')
print('Model Saved')

Model Saved
